<a href="https://colab.research.google.com/github/TushantMan/Fall-Detection-App/blob/docker/Fall_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
from google.colab import drive
drive.mount('/content/drive')

# Set the working directory
import os
SKELETON_DIR = '/content/drive/MyDrive/rudder_preTrainModel_with_testset'
os.chdir(SKELETON_DIR)
! mkdir -p "$SKELETON_DIR/saved_models"
! mkdir -p "$SKELETON_DIR/logs"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
# Set up auto-reloading modules from the working directory
%load_ext autoreload
%autoreload 2

# Install extra dependencies
!pip install -q wandb==0.15.0
!pip install -q torchmetrics==0.11.3

# Set the default figure size
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 120

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
os.chdir(SKELETON_DIR)
!pwd
!ls

/content/drive/MyDrive/rudder_preTrainModel_with_testset
 dataset.py		    Instructions.docx		 'Predict Labels.csv'   test.py
 Dataset_test		    logs			  __pycache__	        Untitled14.ipynb
'Ground Truth Labels.csv'   model_with_architecture.pth   saved_models


In [48]:
import dataset
import test



In [49]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import numpy as np
from itertools import groupby
from sklearn import metrics
import pandas as pd
from dataset import radar_dataset

class FlexibleDummyModel(nn.Module):
    def __init__(self, num_classes=4):
        super(FlexibleDummyModel, self).__init__()
        self.conv = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.fc = nn.Linear(32 * 32 * 31, 32 * num_classes)
        self.num_classes = num_classes

    def forward(self, x):
        print(f"Input shape: {x.shape}")
        if len(x.shape) == 5:
            batch_size, num_images, c, h, w = x.size()
            x = x.view(batch_size * num_images, c, h, w)
        elif len(x.shape) == 4:
            batch_size, c, h, w = x.size()
            num_images = 1
        # Handle the case with 6 dimensions
        elif len(x.shape) == 6:
            batch_size, _, num_images, c, h, w = x.size()
            x = x.view(batch_size * num_images, c, h, w)
        else:
            raise ValueError(f"Unexpected input shape: {x.shape}")

        x = self.conv(x)
        print(f"After conv shape: {x.shape}")
        x = x.view(batch_size, num_images, -1)
        x = x.mean(dim=1)
        print(f"Before fc shape: {x.shape}")
        x = self.fc(x)
        print(f"After fc shape: {x.shape}")
        return x.view(batch_size, 32, self.num_classes)

def run_test_loop(model, testloader, criterion, device):
    model.eval()
    total = 0
    correct = 0
    test_labels = []
    test_predictions = []

    with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            images_test, labels_test = data
            print(f"Batch {i}: images shape: {images_test[0].shape}, labels shape: {labels_test.shape}")

            # Handle the case where images_test is a list and convert to a tensor
            if isinstance(images_test, list):
                images_test = torch.stack(images_test, dim=1).float() # stack along the second dimension (num_images)
            else:
                images_test = images_test.float()

            images_test, labels_test = images_test.to(device), labels_test.to(device)

            # Forward pass
            pred_test = model(images_test)
            pred_test = pred_test.permute(1, 0, 2)

            _, max_index = torch.max(pred_test, dim=2)

            for k in range(images_test.size(0)):
                raw_prediction = list(max_index[:, k].cpu().numpy())
                prediction = [c for c, _ in groupby(raw_prediction) if c != 0]
                prediction = prediction[:15]  # Truncate to match label length
                prediction += [0] * (15 - len(prediction))  # Pad if necessary

                label = labels_test[k].cpu().numpy()
                correct += sum(p == l for p, l in zip(prediction, label))
                test_predictions.extend(prediction)
                test_labels.extend(label)
                total += 15

            # Debug print
            if i == 0:
                print(f"Sample prediction: {prediction}")
                print(f"Sample label: {label}")

            # Break after first batch for debugging
            # break  # Remove this line if you want to run on the full test set

    test_accuracy = 100 * correct / total
    print(f"\nAccuracy on test data: {test_accuracy:.2f}%")

    # More debug information
    print(f"Unique values in predictions: {np.unique(test_predictions)}")
    print(f"Unique values in labels: {np.unique(test_labels)}")

    pd.DataFrame(test_labels).to_csv('Ground Truth Labels.csv')
    pd.DataFrame(test_predictions).to_csv('Predict Labels.csv')

    test_acc = metrics.accuracy_score(test_labels, test_predictions)
    recall = metrics.recall_score(test_labels, test_predictions, average='macro')
    precision = metrics.precision_score(test_labels, test_predictions, average='macro')
    f1 = metrics.f1_score(test_labels, test_predictions, average='macro')

    print(f"Test Accuracy: {test_acc:.4f}")
    print(f"Test Recall: {recall:.4f}")
    print(f"Test Precision: {precision:.4f}")
    print(f"Test F1 Score: {f1:.4f}")

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    im_size_h, im_size_w = 32, 31
    vgg_data_transform = transforms.Compose([
        transforms.Resize((im_size_h, im_size_w)),
        transforms.ToTensor(),
    ])

    test_dir = '/content/drive/MyDrive/rudder_preTrainModel_with_testset/Dataset_test'  # Adjust this path if needed
    testset = radar_dataset(test_dir, 'test', ['radar1_0'], transform=vgg_data_transform)
    testloader = DataLoader(testset, batch_size=1, shuffle=False, num_workers=0)

    # Check the label range in your dataset
    all_labels = [label for _, label in testset]
    unique_labels = torch.unique(torch.cat(all_labels))
    num_classes = len(unique_labels)
    print(f"Number of unique classes in dataset: {num_classes}")
    print(f"Unique labels: {unique_labels}")

    model = FlexibleDummyModel(num_classes=num_classes).to(device)
    criterion = nn.CTCLoss(reduction='mean', zero_infinity=True, blank=0)

    run_test_loop(model, testloader, criterion, device)

Using device: cpu
Number of unique classes in dataset: 4
Unique labels: tensor([1, 6, 7, 8])
Batch 0: images shape: torch.Size([1, 1, 3, 32, 31]), labels shape: torch.Size([1, 15])
Input shape: torch.Size([1, 1, 1, 3, 32, 31])
After conv shape: torch.Size([1, 32, 32, 31])
Before fc shape: torch.Size([1, 31744])
After fc shape: torch.Size([1, 128])
Sample prediction: [3, 3, 1, 2, 1, 2, 3, 2, 1, 2, 1, 1, 1, 2, 0]
Sample label: [1 1 1 1 1 7 7 7 7 7 1 1 1 1 1]
Batch 1: images shape: torch.Size([1, 1, 3, 32, 31]), labels shape: torch.Size([1, 15])
Input shape: torch.Size([1, 1, 1, 3, 32, 31])
After conv shape: torch.Size([1, 32, 32, 31])
Before fc shape: torch.Size([1, 31744])
After fc shape: torch.Size([1, 128])
Batch 2: images shape: torch.Size([1, 1, 3, 32, 31]), labels shape: torch.Size([1, 15])
Input shape: torch.Size([1, 1, 1, 3, 32, 31])
After conv shape: torch.Size([1, 32, 32, 31])
Before fc shape: torch.Size([1, 31744])
After fc shape: torch.Size([1, 128])
Batch 3: images shape: to

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



1. Dataset and Model Setup:
   - It imports necessary libraries and the custom `radar_dataset` class.
   - Defines a `FlexibleDummyModel` class that can handle different input shapes.
   - Sets up the device (CPU or GPU) for computation.

2. Data Preparation:
   - Creates a data transformation pipeline using `torchvision.transforms`.
   - Loads the test dataset using the custom `radar_dataset` class.
   - Creates a DataLoader for batch processing.

3. Model Initialization:
   - Analyzes the dataset to determine the number of unique classes.
   - Initializes the `FlexibleDummyModel` with the correct number of classes.

4. Test Loop (run_test_loop function):
   - Iterates through the test dataset batch by batch.
   - For each batch:
     - Preprocesses the input data (handling different shapes).
     - Passes the data through the model to get predictions.
     - Compares predictions with ground truth labels.
     - Accumulates correct predictions and total samples.

5. Metrics Calculation:
   - Calculates overall accuracy.
   - Uses scikit-learn to compute additional metrics:
     - Accuracy
     - Recall
     - Precision
     - F1 Score

6. Output and Logging:
   - Prints the shape of input data and intermediate results for debugging.
   - Displays a sample prediction and its corresponding label.
   - Prints the calculated metrics.
   - Saves the ground truth labels and predictions to CSV files.

Key Points:
- The script is designed to work with a custom radar dataset, likely containing multiple images per sample.
- It uses a dummy model (not trained) to process the data, so the actual performance metrics are not expected to be meaningful.
- The main purpose is to ensure that data can be correctly loaded, processed through a model, and evaluated, setting up a framework for when you use a real, trained model.

This script essentially serves as a diagnostic tool and a starting point for your radar data classification task. It allows you to verify that:
1. Your dataset is loaded correctly.
2. The data can be properly fed into a neural network model.
3. Predictions can be generated and compared against ground truth labels.
4. Performance metrics can be calculated.

Later, we will replace the dummy model with our actual trained model (model_with_architecture.pth) to get meaningful classification results.